In [4]:
import instructor
import vertexai
from vertexai.generative_models import GenerativeModel, Part
from pydantic import BaseModel
from IPython import display
from typing import Literal
import pandas as pd

In [5]:
vertexai.init(location="europe-west4")

client = instructor.from_vertexai(
    client=GenerativeModel("gemini-2.0-flash"),
    mode=instructor.Mode.VERTEXAI_TOOLS,
)

In [6]:
class AccidentVictim(BaseModel):
    age: int
    gender: str
    occupation: str
    injury: str
    body_affected: Literal["ears", "eyes", "legs", "arms", "head", "shoulder"]
    category: Literal["accident", "crime", "natural_disaster"]

In [22]:
accident_descriptions = pd.Series(
    [
        (
            "A 30 year old man was injured in a car accident. He was driving his car when another car hit him from the side. "
            "He was taken to the hospital with a broken leg. "
            "The man works as a lawyer."
        ),
        (
            "I was walking down the street when I saw a woman fall off her bike. She was taken to the hospital with a broken arm. "
            "She was wearing a helmet, so she didn't hit her head. "
            "The accident happened on a busy street, so there were many witnesses. "
            "To make a living she works as a surgeon."
        ),
        (
            "An earthquake hit the city, causing a building to collapse. "
            "The building was a school, and many children were inside. "
            "A child of 10 years old was injured in the accident. "
            "The boy still went to school and was in the 5th grade. "
            "He injured his shoulder. "
        )
    ]
)
accident_descriptions

0    A 30 year old man was injured in a car acciden...
1    I was walking down the street when I saw a wom...
2    An earthquake hit the city, causing a building...
dtype: object

In [23]:
def extract_accident_victim(accident_description: str) -> AccidentVictim:
    return client.create(
        messages=[
            accident_description,
        ],
        response_model=AccidentVictim,
    )

accident_victims = accident_descriptions.apply(extract_accident_victim)
accident_victims

0    age=30 gender='male' occupation='lawyer' injur...
1    age=30 gender='female' occupation='surgeon' in...
2    age=10 gender='male' occupation='student' inju...
dtype: object

In [24]:
pd.DataFrame([victim.model_dump() for victim in accident_victims])

,age,gender,occupation,injury,body_affected,category
0,30,male,lawyer,broken leg,legs,accident
1,30,female,surgeon,broken arm,arms,accident
2,10,male,student,Shoulder dislocation,shoulder,natural_disaster
